In [1]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G


# Loading Data

In [2]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_tic-tac-toe') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,class
0,NaN,NaN,NaN,NaN,o,o,NaN,o,o,positive
1,NaN,NaN,NaN,NaN,o,o,o,NaN,o,positive
2,NaN,NaN,NaN,NaN,o,o,o,o,NaN,positive
3,NaN,NaN,x,NaN,NaN,o,o,NaN,b,positive
4,NaN,x,NaN,x,NaN,NaN,b,NaN,b,positive
...,...,...,...,...,...,...,...,...,...,...
953,o,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,negative
954,NaN,NaN,o,x,NaN,NaN,NaN,NaN,NaN,negative
955,o,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,negative
956,o,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,negative


# Dataset Analyses

In [3]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

Number of features  = 9


In [4]:
print(f"Does # of features = d  = {not S.dropna().empty}")

Does # of features = d  = False


In [5]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

d = 5


In [6]:
P = S
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AR = {len(B)}")

Length of Node Cover for AR = 5


In [7]:
P = R_SR(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for SR = {len(B)}")

Length of Node Cover for SR = 5


In [8]:
P = R_AD(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AD = {len(B)}")

Length of Node Cover for AD = 5


In [9]:
P = S
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AR = {len(B)}")

Length of Node Cover greedy for AR = 3


In [10]:
P = R_SR(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for SR = {len(B)}")

Length of Node Cover greedy for SR = 3


In [11]:
P = R_AD(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AD = {len(B)}")

Length of Node Cover greedy for AD = 3


# Combinations

In [12]:
column_values = []

for column in S.columns[:-1]: #ignoring class column
    column_values.append(list(S[column].dropna().unique()))
    
# All possible combinations    
combinations = list(itertools.product(*column_values))

print('Number of Possible combinations =', len(combinations))

Number of Possible combinations = 19683


In [13]:
# # calculate 10% sample size
# sample_size = int(len(combinations)*0.1)

# # Get a random sapmple of combinations
# combinations = random.sample(combinations, sample_size)

# print('Number of Possible combinations after sampling =', len(combinations))

# Node Cover

In [14]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))
print("Running time =", end_time - start_time)

100%|██████████████████████████████████| 19683/19683 [76:00:56<00:00, 13.90s/it]

NC AD
(Max Depth, Min Depth, Average Depth) = (9, 5, 8.324340801707057)
Running time = 273656.8166208267
